In [93]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os
import time
from datetime import datetime, timedelta
import pandas as pd
import keybert
from nltk.corpus import stopwords
import re 


In [10]:
load_dotenv()
user = os.getenv("UID")
password = os.getenv("PASSWORD")

#JOB TYPE: 
# Graduate 
# Internsip 
# Temporary 
# Summer
job_type = "Internship"

<h2>Logging into the Job Portal</h2>

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.cedars.hku.hk/netjobs")
main_page = driver.current_window_handle

# Click on Student Login
student_login = driver.find_element(By.XPATH, "//a[text()='HKU Student']")
student_login.click()

#Input email and log in
email_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "email"))
)
login_button = driver.find_element(By.ID, "login_btn")

email_input.send_keys(user)
login_button.click()

#Input password
password_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "passwordInput"))
)
sign_in_button = driver.find_element(By.ID, "submitButton")
password_input.send_keys(password)
sign_in_button.click()

#Trust Page
continue_button = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "idSIButton9"))
)
continue_button.click()
#Stay Singed in page
stay_button = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='submit' and @id='idSIButton9']"))
)
stay_button.click()

check_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='checkbox']"))
)
check_box.click()
agree_btn = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.ID, "btn-agree"))
)
agree_btn.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

In [19]:

yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')
yesterday

'2025-07-18'

In [ ]:
def clean_data(data):
    # data: list of strings corresponding to job details
    for i in range(len(data)):
        data[i] = data[i].split(':\n')
    return data

def get_data(driver):
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    for window_handle in driver.window_handles:
        if window_handle != main_page:
            driver.switch_to.window(window_handle)
            break
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='content']"))
    )
    extract_data = driver.find_elements(By.XPATH, "//div[@id='content']//div[contains(@class, 'crow')]")
    data = [i.text for i in extract_data if i.text.strip()]
    
    driver.close()
    driver.switch_to.window(main_page)
    return data

details = []
jobs = driver.find_elements(By.XPATH, "//table[@id='search_jobs']/tbody/tr")

for i, job in enumerate(jobs):
    job.click()
    data = clean_data(get_data(driver))
    details.append(data)

driver.close()

In [23]:
details[2]

[['Job ID', 'G2500061'],
 ['Posting Date', '2025-07-16'],
 ['Employment Type', 'Internship'],
 ['Employment Mode', 'Full-time'],
 ['Company Name (Eng)', 'DRW Associates Hong Kong Limited'],
 ['Nature of Business', 'Banking / Financial Services'],
 ['Origin of Company Ownership', 'United States'],
 ['No. of Employees', '1000-5000 employees'],
 ['Position Offered', 'Quantitative Research Intern'],
 ['No. of Vacancies', '4'],
 ['Job Nature', 'Others'],
 ['Job Description',
  "Our formula for success is to hire exceptional people, encourage their ideas and reward their results.\nAs a Quantitative Research Intern you will have an opportunity to solve challenging problems arising in a trading environment while utilizing the latest statistical scientific algorithms, machine learning techniques and derivatives pricing theory.  The teams focus on non-latency sensitive investment opportunities and multi-asset class derivatives strategies across geographies. Our teams emphasize cutting-edge innov

In [24]:
job_details = dict()
for job in details:
    for detail in job:
        if detail[0] not in job_details:
            job_details[detail[0]] = 1
        else:
            job_details[detail[0]] += 1

job_details


{'Job ID': 20,
 'Posting Date': 20,
 'Employment Type': 20,
 'Employment Mode': 20,
 'Company Name (Eng)': 20,
 'Company Name (Chi)': 10,
 'Nature of Business': 20,
 'Origin of Company Ownership': 20,
 'No. of Employees': 20,
 'Position Offered': 20,
 'No. of Vacancies': 20,
 'Job Nature': 20,
 'Job Description': 20,
 'Basic Salary': 20,
 'Work Location': 20,
 'Work Mode': 20,
 'Fields of Study Required': 20,
 'Level of Award ': 20,
 'No. of working hours per day': 20,
 'No. of working days per week': 20,
 'Closing Date for Application': 20,
 'Application should be submitted by/via:': 20,
 'Email': 11,
 'Application Documents Required': 20,
 'Online via NETjobs:': 4,
 'Other Benefits': 3,
 'Online Application at company website': 8,
 'Application should be addressed to': 3}

In [25]:
jobs_data = []
for job in details:
    jobs_dict = dict()
    for detail in job:
        if len(detail) == 2: 
            jobs_dict[detail[0]] = detail[1]
        elif len(detail) > 2:  
            jobs_dict[detail[0]] = ' '.join(detail[1:])
        elif len(detail) == 1:
            if detail[0] not in jobs_dict:
                jobs_dict[detail[0]] = 1
            else:
                jobs_dict[detail[0]] += 1
    jobs_data.append(jobs_dict)
jobs_df = pd.DataFrame(jobs_data)

In [26]:
# jobs_df.to_json("data.json", orient="records", indent=4)

jobs_df.columns

Index(['Job ID', 'Posting Date', 'Employment Type', 'Employment Mode',
       'Company Name (Eng)', 'Company Name (Chi)', 'Nature of Business',
       'Origin of Company Ownership', 'No. of Employees', 'Position Offered',
       'No. of Vacancies', 'Job Nature', 'Job Description', 'Basic Salary',
       'Work Location', 'Work Mode', 'Fields of Study Required',
       'Level of Award ', 'No. of working hours per day',
       'No. of working days per week', 'Closing Date for Application',
       'Application should be submitted by/via:', 'Email',
       'Application Documents Required', 'Online via NETjobs:',
       'Other Benefits', 'Online Application at company website',
       'Application should be addressed to'],
      dtype='object')

In [92]:
print(jobs_df['Job Description'].iloc[19])

"""
About Us
Role Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities
Key Responsibilities
Requirements/Qualifications
"""

Apply your experience and knowledge in areas of accelerated computing and machine learning (esp. AI4S, including weather, CFD, Quantum, materials/chemistry, Robotic, Life science).
Conduct AI engineering via playing with software and hardware and collaborating with external researchers. Partner with internal groups to transfer technologies and innovate products.
Assist in building AIGC, AI4S tools with the state-of-the-art AI models. It can also help the AI community use multiple NVIDIA SDKs or frameworks (esp. PhysicsNeMo/Modulus/CUDA-Q/Isaac/BioNeMo) to realize fantastic idea!
Mainly works in Beijing/Shanghai/Shenzhen office, 3~4 days/week with at least 6 months available.


'\nAbout Us\nRole Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities\nKey Responsibilities\nRequirements/Qualifications\n'

In [ ]:
def clean_desc(desc):
    desc = re.split(r'', desc)
    return desc

In [67]:
def get_tags(input_df:pd.DataFrame, model:keybert) -> list:
    tags = []
    job_nature = input_df['Job Nature'].replace('/', ' ') 
    job_desc = clean_desc(input_df['Job Description'])
    pos_offered = input_df['Position Offered'] 
    nature_of_business = input_df['Nature of Business'] 

    tags.append(model.extract_keywords(job_desc, stop_words=stopwords.words('english')))
    return tags

In [57]:
model = keybert.KeyBERT()

In [73]:
input_data = jobs_df[['Job ID', 'Job Nature', 'Job Description', 'Position Offered', 'Nature of Business']]
print(input_data.iloc[1]['Job Description'])
print(get_tags(input_data.iloc[1], model))

Role Overview
We are seeking a proactive and technically-inclined Business Analyst to join our innovation team. In this role, you will be the vital link between our business stakeholders and engineering teams, driving the development of proof-of-concept (POC) and Mobile APP projects from initial idea to functional prototype. You will be responsible for analyzing business needs, defining clear requirements, and ensuring that the proposed technical solutions align with strategic goals. Your ability to understand and articulate both business objectives and technical possibilities will be key to our success in rapidly testing and iterating on new, innovative solutions.

Key Responsibilities
Requirements Analysis: Collaborate with product managers, designers, and business stakeholders to elicit, analyze, and refine requirements for new product features and POC projects.
Technical Liaison: Act as the primary bridge between business users and the development team, translating business needs i

Job ID  
Company Name  
Nature of Business  
Position Offered  
Job Nature
Job Description

Data types: NULL INTEGER REAL TEXT BLOB

In [ ]:
#Set up a connection
# conn = sqlite3.connect('jobs.db')
# cursor = conn.cursor()
# create_table_query = """
# CREATE TABLE IF NOT EXISTS jobs (
#     JobID TEXT PRIMARY KEY,
#     PostingDate TEXT,
#     EmploymentType TEXT,
#     EmplymentMode TEXT,
#     CompanyNameEng TEXT,
#     CompanyNameChi TEXT, 
#     NatureOfBusiness TEXT,
#     OriginOfCompanyOwnership TEXT,
#     NoOfEmployees TEXT,
#     PositionOffered TEXT,
#     NoOfVacancies TEXT,
#     JobNature TEXT,
#     JobDescription TEXT,
#     BasicSalary TEXT,
#     WorkLocatin TEXT, 
#     WorkMode TEXT,
#     FieldsOfStudyRequired TEXT,
#     LevelOfAward TEXT,
#     NoOfWorkingHoursPerDay INTEGER,
#     NoOfWorkingHoursPerWeek INTEGER,
#     ClosingDataForApplication TEXT,
#     ApplicationShouldBeSubmittedVia TEXT,
#     ApplicationDocumentsRequired TEXT
# );
# """

# cursor.execute(create_table_query)

In [ ]:


import nbformat

# Load the notebook
with open('get_jobs.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Extract code cells
code_cells = [cell.source for cell in notebook.cells if cell.cell_type == 'code']

# Write to a .py file
with open('output_script.py', 'w') as f:
    for cell in code_cells:
        f.write(cell + '\n\n')
